In [2]:
import cv2
import os
import numpy as np


def inject_gaussian(image, mean = 0, sigma = 25):
    normalised_image = image.astype(np.float32) / 255.0 
    gaussian = np.random.normal(mean, sigma / 255.0, normalised_image.shape)
    noise = normalised_image + gaussian
    alt_image = np.clip(noise * 255, 0, 255).astype(np.uint8) 
    return alt_image

def inject_poisson(image):
    scaled_image = image.astype(np.float32) / 255.0 * 100.0 
    noise = np.random.poisson(scaled_image) / 100.0

    alt_image = np.clip(noise * 255, 0, 255).astype(np.uint8) 
    return alt_image

def inject_gaussian_noise(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)

            noisy_image = inject_gaussian(image)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, noisy_image)

            print(f"Processed and saved: {output_path}")

def inject_poisson_noise(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)

            noisy_image = inject_poisson(image)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, noisy_image)

            print(f"Processed and saved: {output_path}")

def inject_both(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)

            poisson_image = inject_poisson(image)
            noisy_image = inject_gaussian(poisson_image, sigma = 25)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, noisy_image)

            print(f"Processed and saved: {output_path}")

    
input_folder = 'images/finetune'
output_folder = 'noisy_output/finetune'
gaussian_folder = 'gaussian_output/val'
poisson_folder = 'poisson_output/val'

# inject_gaussian_noise(input_folder, gaussian_folder)
# inject_poisson_noise(input_folder, poisson_folder)
inject_both(input_folder, output_folder)

Processed and saved: noisy_output/finetune\finetune_img_0000_000.png
Processed and saved: noisy_output/finetune\finetune_img_0000_001.png
Processed and saved: noisy_output/finetune\finetune_img_0000_002.png
Processed and saved: noisy_output/finetune\finetune_img_0000_003.png
Processed and saved: noisy_output/finetune\finetune_img_0000_004.png
Processed and saved: noisy_output/finetune\finetune_img_0000_005.png
Processed and saved: noisy_output/finetune\finetune_img_0000_006.png
Processed and saved: noisy_output/finetune\finetune_img_0000_007.png
Processed and saved: noisy_output/finetune\finetune_img_0000_008.png
Processed and saved: noisy_output/finetune\finetune_img_0000_009.png
Processed and saved: noisy_output/finetune\finetune_img_0000_010.png
Processed and saved: noisy_output/finetune\finetune_img_0000_011.png
Processed and saved: noisy_output/finetune\finetune_img_0000_012.png
Processed and saved: noisy_output/finetune\finetune_img_0000_013.png
Processed and saved: noisy_output/

# Bit for Pipeline.
Denoise -> Deblur

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

task    = 'Denoising'
inp_dir = 'doubleblur_noise/images/train'
out_dir = 'doubleblur_noise_denoise/images/train'

os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "model_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.jpg')), restored)

print(f"Files saved at {out_dir}")

Files saved at doubleblur_noise_denoise/images/train


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] 
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

task    = 'Denoising'
inp_dir = 'doubleblur_noise/images/val'
out_dir = 'doubleblur_noise_denoise/images/val'

os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "model_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.jpg')), restored)

print(f"Files saved at {out_dir}")

Files saved at doubleblur_noise_denoise/images/val


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

task    = 'Deblurring'
inp_dir = 'doubleblur_noise_denoise/images/train'
out_dir = 'doubleblur_noise_denoise_deblur/images/train'

os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "model_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.jpg')), restored)

print(f"Files saved at {out_dir}")

Files saved at doubleblur_noise_denoise_deblur/images/train


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:]
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

task    = 'Deblurring'
inp_dir = 'doubleblur_noise_denoise/images/val'
out_dir = 'doubleblur_noise_denoise_deblur/images/val'

os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "model_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.jpg')), restored)

print(f"Files saved at {out_dir}")

Files saved at doubleblur_noise_denoise_deblur/images/val
